## Dataframe Transformations in PySpark

### 1. Selecting Columns
The most basic transformation is selecting specific columns from a DataFrame.

You can select data from a DataFrame using various methods and techniques, such as selecting specific columns, applying conditions, or using SQL queries.

In [0]:
# creating a datafrae
df = spark.read.format("csv") \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .load("dbfs:/FileStore/shared_uploads/zaderohish5@gmail.com/employee_1.csv")

# output of df
df.show(5)

+---+--------+---+------+------------+--------+
| id|    name|age|salary|     address| nominee|
+---+--------+---+------+------------+--------+
|  1|  Rohish| 26| 75000|        Pune|nominee1|
|  2|  Nikita| 23|100000|uttarpradesh|nominee2|
|  3|  Pritam| 22|150000|   Bangalore|nominee3|
|  4|Prantosh| 17|200000|     Kolkata|nominee4|
|  5|  Vikash| 31|300000|        null|nominee5|
+---+--------+---+------+------------+--------+



**To select specific columns from a DataFrame, use the .select() method.**

In [0]:
df.select("name").show()

+--------+
|    name|
+--------+
|  Rohish|
|  Nikita|
|  Pritam|
|Prantosh|
|  Vikash|
+--------+



In [0]:
df.select("name", "salary", "*").show()

+--------+------+---+--------+---+------+------------+--------+
|    name|salary| id|    name|age|salary|     address| nominee|
+--------+------+---+--------+---+------+------------+--------+
|  Rohish| 75000|  1|  Rohish| 26| 75000|        Pune|nominee1|
|  Nikita|100000|  2|  Nikita| 23|100000|uttarpradesh|nominee2|
|  Pritam|150000|  3|  Pritam| 22|150000|   Bangalore|nominee3|
|Prantosh|200000|  4|Prantosh| 17|200000|     Kolkata|nominee4|
|  Vikash|300000|  5|  Vikash| 31|300000|        null|nominee5|
+--------+------+---+--------+---+------+------------+--------+



In [0]:
df.select(df.name, df.age).show()

+--------+---+
|    name|age|
+--------+---+
|  Rohish| 26|
|  Nikita| 23|
|  Pritam| 22|
|Prantosh| 17|
|  Vikash| 31|
+--------+---+



In [0]:
# Using Col method
from pyspark.sql.functions import *

df.select(col("name"), col("age")).show()


+--------+---+
|    name|age|
+--------+---+
|  Rohish| 26|
|  Nikita| 23|
|  Pritam| 22|
|Prantosh| 17|
|  Vikash| 31|
+--------+---+



**Why we need `col()` method for selecting when we can directly select with column names in select()**

Lets try to do some expression. lets add 5 in id column

In [0]:
# Lets try to do some expression. lets add 5 in id column
df.select("id + 5")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-3222529370696004>:2
      1 # Lets try to do some expression. lets add 5 in id column
----> 2 df.select("id + 5")

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:3023, in DataFrame.select(self, *cols)
   2978 def select(self, *cols: "ColumnOrName") -> "DataFrame":  # type: ignore[misc]
   2979     """Projects a set of expressions and returns a new :class:`DataFrame`.
   2980 
   2981     .. versionadded:: 1.3.0
   (...)
   3021     +-----+---+
   30

You can see the error, spark is taking `id + 5` as a column and looking into the DF but `id + 5` doesnt exist.

When you pass "id + 5" as a string inside .select(), PySpark interprets it as a literal string rather than an actual operation on a column. PySpark does not understand that "id + 5" means "take the id column and add 5 to its values." Instead, it treats "id + 5" as just a plain string.

**Correct Way to Perform the Transformation:**
To correctly add 5 to the id column in PySpark, you need to explicitly reference the column using `col()` and then perform the addition operation.

In [0]:
# Lets try to do some expression. lets add 5 in id column
df.select("id", col("id") + 5).show()

+---+--------+
| id|(id + 5)|
+---+--------+
|  1|       6|
|  2|       7|
|  3|       8|
|  4|       9|
|  5|      10|
+---+--------+



**selectExpr() for SQL-like Syntax:**

If you want to use a more SQL-like syntax, you can use selectExpr(), which allows you to pass expressions in string format, but with the understanding that you are writing SQL-like expressions.

In [0]:
df.selectExpr("name", "age", "salary", "id + 5").show()

+--------+---+------+--------+
|    name|age|salary|(id + 5)|
+--------+---+------+--------+
|  Rohish| 26| 75000|       6|
|  Nikita| 23|100000|       7|
|  Pritam| 22|150000|       8|
|Prantosh| 17|200000|       9|
|  Vikash| 31|300000|      10|
+--------+---+------+--------+



In [0]:

# we can write sql like queries
df.select(expr("id as emp_id"), expr("id + 5 as new_emp_id")).show()

+------+----------+
|emp_id|new_emp_id|
+------+----------+
|     1|         6|
|     2|         7|
|     3|         8|
|     4|         9|
|     5|        10|
+------+----------+



**Pandas way to select columns in PySpark**

- when you have multiple DataFrames with duplicate column names, it can be difficult to distinguish between those columns when performing operations like joining 
- Selecting in pandas way is helpful when we have same columns names in two or more dataframes to distinct the columns

In [0]:
df.select(df["name"], df["age"]).show()

+--------+---+
|    name|age|
+--------+---+
|  Rohish| 26|
|  Nikita| 23|
|  Pritam| 22|
|Prantosh| 17|
|  Vikash| 31|
+--------+---+



**Spark SQL**

To use the spark.sql() method in PySpark, you must create a `temporary view` or `global temporary view` from a DataFrame.

you can create a temporary view using the `.createOrReplaceTempView()` method, which makes the DataFrame available to SQL queries using spark.sql().

In [0]:
df.createOrReplaceTempView("employee_vw")

In [0]:
spark.sql("""
        select * from employee_vw          
""").show()

+---+--------+---+------+------------+--------+
| id|    name|age|salary|     address| nominee|
+---+--------+---+------+------------+--------+
|  1|  Rohish| 26| 75000|        Pune|nominee1|
|  2|  Nikita| 23|100000|uttarpradesh|nominee2|
|  3|  Pritam| 22|150000|   Bangalore|nominee3|
|  4|Prantosh| 17|200000|     Kolkata|nominee4|
|  5|  Vikash| 31|300000|        null|nominee5|
+---+--------+---+------+------------+--------+



### Types of Views

**Temporary View (createOrReplaceTempView):**
- A temporary view is session-scoped and will be dropped when the Spark session ends.
- It can be created using `.createOrReplaceTempView().`
- Example: `df.createOrReplaceTempView("temp_view")`

**Global Temporary View (createGlobalTempView):**
- A global temporary view is shared across all Spark sessions and will exist until the Spark application is terminated.
- It can be created using `.createGlobalTempView().`
- Example: `df.createGlobalTempView("global_temp_view")`
- To query a global temporary view, you must use the special namespace `global_temp:`
- `result = spark.sql("SELECT * FROM global_temp.global_temp_view")`


